<a href="https://colab.research.google.com/github/tushitgarg/Hate-Speech-Content-Moderation/blob/master/Quora_Glove.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import json
import pymongo
from pymongo import MongoClient
import pprint as pp
! pip install normalise
import nltk
nltk.download('names')
nltk.download('brown')

import numpy as np
import multiprocessing as mp

import string
import spacy 
import en_core_web_sm
from nltk.tokenize import word_tokenize
from sklearn.base import TransformerMixin, BaseEstimator
from normalise import normalise
import re

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Flatten, LSTM, Conv1D, MaxPooling1D, Dropout, Activation,GRU, SimpleRNN
from keras.layers.embeddings import Embedding
from keras import layers
from keras.initializers import Constant


     |████████████████████████████████| 15.7MB 250kB/s 
[nltk_data] Downloading package names to /root/nltk_data...
[nltk_data]   Unzipping corpora/names.zip.
[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Unzipping corpora/brown.zip.


/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.semi_supervised.label_propagation module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.semi_supervised. Anything that cannot be imported from sklearn.semi_supervised is now part of the private API.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator LabelPropagation from version 0.18 when using version 0.22.2.post1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
Using TensorFlow backend.


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
cd '/content/drive/My Drive/minor2/Hate-Speech-Content-Moderation/'

/content/drive/My Drive/minor2/Hate-Speech-Content-Moderation


In [4]:
df1=pd.read_excel('quora.xlsx')
#df1.drop(['level_0','index','Unnamed: 0'],axis=1,inplace=True)
df1.head(10)

,qid,text,label
0,8eeb2fa6a60d93c5ce3c,"What is the difference between real, true, act...",0
1,420689e2da77a9254362,Can an applicant with JEST score in PhD merit ...,0
2,b4d8ce47a727326a8916,How can I get the syllabus of JEE Advanced for...,0
3,197be9aa5fbef88659ee,What is fee structure of KLE Sheshagiri Colleg...,0
4,a05cc194d3952fce5856,How do I pick few wines that will go with food...,0
5,e1ddc63412371760c4f8,Where can I find a list of products that use S...,0
6,4ceb54eb7d9d41e447b8,Did you ever watch your wife have sex?,0
7,911a6777272ef36f45f4,What are door and window hardware commonly use...,0
8,3038fbfa8d74e8e57c9d,Why do we cancel a product purchased online?,0
9,b2ae47cfd86ae994440a,Do people who speak no languages think in pict...,0


In [5]:
np.mean(df1['label'])

0.29186533737899145

In [0]:
text=df1['text']
labels=df1['label']

In [0]:
nlp = en_core_web_sm.load()


class TextPreprocessor(BaseEstimator, TransformerMixin):
    def __init__(self,
                 variety="BrE",
                 user_abbrevs={},
                 n_jobs=1):
        """
        Text preprocessing transformer includes steps:
            1. Text normalization
            2. Punctuation removal
            3. Stop words removal
            4. Lemmatization
        
        variety - format of date (AmE - american type, BrE - british format) 
        user_abbrevs - dict of user abbreviations mappings (from normalise package)
        n_jobs - parallel jobs to run
        """
        self.variety = variety
        self.user_abbrevs = user_abbrevs
        self.n_jobs = n_jobs

    def fit(self, X, y=None):
        return self

    def transform(self, X, *_):
        X_copy = X.copy()
        return X_copy.apply(self._preprocess_text)

    def _preprocess_part(self, part):
        return part.apply(self._preprocess_text)

    def _preprocess_text(self, text):
        text=self._clean_text(text)
        normalized_text = self._normalize(text)
        doc = nlp(normalized_text)
        removed_punct = self._remove_punct(doc)
        removed_stop_words = self._remove_stop_words(removed_punct)
        return self._lemmatize(removed_stop_words)

    def _normalize(self, text):
        # some issues in normalise package
        try:
            return ' '.join(normalise(text, variety=self.variety, user_abbrevs=self.user_abbrevs, verbose=False))
        except:
            return text

    def _remove_punct(self, doc):
        return [t for t in doc if t.text not in string.punctuation]

    def _remove_stop_words(self, doc):
        return [t for t in doc if not t.is_stop]

    def _lemmatize(self, doc):
        return ' '.join([t.lemma_ for t in doc])
    
    
    def _clean_text(self,text):
      replace_1 = re.compile("(\.)|(\;)|(\:)|(\!)|(\')|(\?)|(\,)|(\")|(\()|(\))|(\[)|(\])")
      replace_2 = re.compile("(<br\s*/><br\s*/>)|(\-)|(\/)")
      try:
        text=re.sub(r"http\S+", "", text)
      except:
        print(text)
      text = replace_1.sub("", text)
      text = replace_2.sub(" ", text)
      text=re.sub('\s+',' ',text)
      return text

In [0]:
%%time
text = TextPreprocessor(n_jobs=-1).transform(df1['text'])

In [0]:
text2=[]
for i in text:
      lst=i.split()
      text2.append(lst)

In [9]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer() 
tokenizer.fit_on_texts(text2) 
sequences = tokenizer.texts_to_sequences(text2)
word_index = tokenizer.word_index
print('Found %s unique tokens.',len(word_index))
from keras.preprocessing.sequence import pad_sequences
maxlen = max([len(s.split()) for s in text])
tweets_pad = pad_sequences(sequences, maxlen=maxlen)
print(tweets_pad.shape) 


Found %s unique tokens. 28110
(13842, 58)


In [0]:
df3=pd.concat([pd.DataFrame(text),pd.DataFrame(tweets_pad)],axis=1)

In [11]:
print('Loading word vectors')
embeddings_index = {} # We create a dictionary of word -> embedding
f = open('/content/drive/My Drive/minor2/twitter2/glove.6B.100d.txt','r') # Open file
# In the dataset, each line represents a new word embedding
# The line starts with the word and the embedding values follow
for line in tqdm(f):
    values = line.split()
    word = values[0] # The first value is the word, the rest are the values of the embedding
    values = np.asarray(values[1:], dtype='float32') # Load embedding
    embeddings_index[word] = values # Add embedding to our embedding dictionary
f.close()

print('Found %s word vectors.' % len(embeddings_index))

0it [00:00, ?it/s]

Loading word vectors


400001it [00:22, 17770.95it/s]

Found 400001 word vectors.


In [12]:
all_embs = np.stack(embeddings_index.values())
emb_mean = all_embs.mean() # Calculate mean
emb_std = all_embs.std() # Calculate standard deviation
emb_mean,emb_std

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2822: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  if self.run_code(code, result):


(0.0044520576, 0.40815717)

In [0]:
embedding_dim = 100
word_index = tokenizer.word_index
num_words = len(word_index)+1
embedding_matrix = np.random.normal(emb_mean, emb_std, (num_words, embedding_dim))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None: 
        embedding_matrix[i] = embedding_vector

In [14]:
embedding_matrix.shape

(28111, 100)

In [0]:
import matplotlib.pyplot as plt
plt.style.use('ggplot')

def plot_history(history,name_of_fig):
    acc = history.history['acc']
    val_acc = history.history['val_acc']
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    x = range(1, len(acc) + 1)

    f=plt.figure(figsize=(12, 5))
    plt.subplot(1, 2, 1)
    plt.plot(x, acc, 'b', label='Training acc')
    plt.plot(x, val_acc, 'r', label='Validation acc')
    plt.title('Training and validation accuracy')
    plt.legend()
    plt.subplot(1, 2, 2)
    plt.plot(x, loss, 'b', label='Training loss')
    plt.plot(x, val_loss, 'r', label='Validation loss')
    plt.title('Training and validation loss')
    plt.legend()
    f.savefig(name_of_fig, bbox_inches='tight', dpi=400)

In [37]:
model = Sequential()
embedding_layer = Embedding(num_words,
                            100,
                            embeddings_initializer = Constant(embedding_matrix),
                            weights = [embedding_matrix],
                            input_length = maxlen,
                            trainable=False)
model.add(embedding_layer)
model.add(layers.Conv1D(128, 5, activation='relu'))
model.add(layers.GlobalMaxPooling1D())
#model.add(layers.Dense(10, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, 58, 100)           2811100   
_________________________________________________________________
conv1d_4 (Conv1D)            (None, 54, 128)           64128     
_________________________________________________________________
global_max_pooling1d_4 (Glob (None, 128)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 129       
Total params: 2,875,357
Trainable params: 64,257
Non-trainable params: 2,811,100
_________________________________________________________________


In [44]:
model = Sequential()
embedding_layer = Embedding(num_words,
                            100,
                            embeddings_initializer = Constant(embedding_matrix),
                            input_length = maxlen,
                            trainable=False)
model.add(embedding_layer)
model.add(LSTM(units=100))
model.add(Dropout(.3))
model.add(layers.Dense(1, activation='sigmoid'))
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_6 (Embedding)      (None, 58, 100)           2811100   
_________________________________________________________________
lstm_2 (LSTM)                (None, 100)               80400     
_________________________________________________________________
dropout_2 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 1)                 101       
Total params: 2,891,601
Trainable params: 80,501
Non-trainable params: 2,811,100
_________________________________________________________________


In [50]:
model = Sequential()
embedding_layer = Embedding(num_words,
                            100,
                            embeddings_initializer = Constant(embedding_matrix),
                            input_length = maxlen,
                            trainable=False)
model.add(embedding_layer)
model.add(GRU(units=100,dropout=0.2 , recurrent_dropout=0.2))
model.add(layers.Dense(1, activation='sigmoid'))
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
model.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_7 (Embedding)      (None, 58, 100)           2811100   
_________________________________________________________________
gru_1 (GRU)                  (None, 100)               60300     
_________________________________________________________________
dense_7 (Dense)              (None, 1)                 101       
Total params: 2,871,501
Trainable params: 60,401
Non-trainable params: 2,811,100
_________________________________________________________________


In [56]:
model = Sequential()
embedding_layer = Embedding(num_words,
                            100,
                            embeddings_initializer = Constant(embedding_matrix),
                            input_length = maxlen,
                            trainable=False)
model.add(embedding_layer)
model.add(Flatten())
model.add(layers.Dense(10, activation='sigmoid'))
#model.add(GRU(units=100,dropout=0.2 , recurrent_dropout=0.2))
model.add(layers.Dense(1, activation='sigmoid'))
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
model.summary()

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_8 (Embedding)      (None, 58, 100)           2811100   
_________________________________________________________________
flatten_1 (Flatten)          (None, 5800)              0         
_________________________________________________________________
dense_8 (Dense)              (None, 10)                58010     
_________________________________________________________________
dense_9 (Dense)              (None, 1)                 11        
Total params: 2,869,121
Trainable params: 58,021
Non-trainable params: 2,811,100
_________________________________________________________________


In [62]:
model = Sequential()
embedding_layer = Embedding(num_words,
                            100,
                            embeddings_initializer = Constant(embedding_matrix),
                            input_length = maxlen,
                            trainable=False)
model.add(embedding_layer)
#model.add(Flatten())
model.add(SimpleRNN(32, dropout=.2, recurrent_dropout=.2))
model.add(layers.Dense(10, activation='sigmoid'))
model.add(layers.Dense(1, activation='sigmoid'))
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
model.summary()

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_9 (Embedding)      (None, 58, 100)           2811100   
_________________________________________________________________
simple_rnn_1 (SimpleRNN)     (None, 32)                4256      
_________________________________________________________________
dense_10 (Dense)             (None, 10)                330       
_________________________________________________________________
dense_11 (Dense)             (None, 1)                 11        
Total params: 2,815,697
Trainable params: 4,597
Non-trainable params: 2,811,100
_________________________________________________________________


In [0]:
from sklearn.model_selection import train_test_split
X_train, X_testing ,y_train, y_testing = train_test_split(df3, labels, test_size = 0.60, random_state = 1)

In [0]:
X_test, X_val ,y_test, y_val = train_test_split(X_testing, y_testing, test_size = 0.50, random_state = 0)

In [65]:
print('Training.....')
history=model.fit(X_train.iloc[:,1:], y_train, batch_size=256,epochs=10,validation_data=(X_val.iloc[:,1:],y_val),verbose=True)

Training.....
Train on 5536 samples, validate on 4153 samples
Epoch 1/10
5536/5536 [==============================] - 1s 218us/step - loss: 1.0847 - accuracy: 0.2896 - val_loss: 0.8869 - val_accuracy: 0.2933
Epoch 2/10
5536/5536 [==============================] - 1s 165us/step - loss: 0.8030 - accuracy: 0.3255 - val_loss: 0.6815 - val_accuracy: 0.6210
Epoch 3/10
5536/5536 [==============================] - 1s 168us/step - loss: 0.6595 - accuracy: 0.6656 - val_loss: 0.6282 - val_accuracy: 0.7058
Epoch 4/10
5536/5536 [==============================] - 1s 161us/step - loss: 0.6227 - accuracy: 0.7083 - val_loss: 0.6147 - val_accuracy: 0.7067
Epoch 5/10
5536/5536 [==============================] - 1s 163us/step - loss: 0.6126 - accuracy: 0.7094 - val_loss: 0.6099 - val_accuracy: 0.7067
Epoch 6/10
5536/5536 [==============================] - 1s 164us/step - loss: 0.6077 - accuracy: 0.7106 - val_loss: 0.6076 - val_accuracy: 0.7067
Epoch 7/10
5536/5536 [==============================] - 1s 162

In [0]:
y_classes=model.predict_classes(X_test.iloc[:,1:])
y_test = y_test.astype(int)

In [42]:
print(len(X_val))
print(len(y_val))
print(df3.shape)
print(len(labels))

4153
4153
(13842, 59)
13842


In [0]:
dataset = pd.DataFrame({'y_test': y_test, 'y_classes': y_classes[:,0]})

In [0]:
1-np.mean(dataset['y_test'])

0.7064772453647965

In [0]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix

In [67]:
# accuracy: (tp + tn) / (p + n)
accuracy = accuracy_score(y_test, y_classes)
print('Accuracy:{}\n'.format(accuracy))
# precision tp / (tp + fp)
precision = precision_score(y_test, y_classes)
print('Precision:{}\n'.format(precision))
# recall: tp / (tp + fn)
recall = recall_score(y_test, y_classes)
print('Recall:{}\n'.format(recall))
# f1: 2 tp / (2 tp + fp + fn)
f1 = f1_score(y_test, y_classes)
print('F1 score:{}\n'.format(f1))
matrix = confusion_matrix(y_test, y_classes)
print("Confusion Matrix:")
print(matrix)

Accuracy:0.7064772453647965

Precision:0.0

Recall:0.0

F1 score:0.0

Confusion Matrix:
[[2934    0]
 [1219    0]]


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
